In [ ]:
import collections
import functools
from imposm.parser import OSMParser
import json
from matplotlib import collections as mc
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import numpy as np
import pandas as pd
import pyproj
import requests
import scipy as sp
import rtree
# import seaborn as sb
from scipy import signal
# import shapely
import shapely.geometry
%pylab inline

import data_munging

### Ride Report Method

Here, we use the `match` method from the OSRM API with the code modified to return only the endpoints of segments. This allows us to aggregate over OSM segments since the node IDs are uniquely associated with a lat/lon pair given sufficient precision in the returned coordinates. The API recommends not using every single value for the match method, but I'm giving them regardless because it's easier to code. Down-sampling the ride might actually help to smooth some of the rides. (or perhaps not if we accidentally get a jagged part).

Currently, I am unsure how to mark up OSM data with bumpiness information, as we have 
data that look like this in the raw OSM file:


``
        <way id="23642309" version="25" timestamp="2013-12-26T23:03:24Z" changeset="19653154" uid="28775" user="StellanL">
                <nd ref="258965973"/>
                <nd ref="258023463"/>
                <nd ref="736948618"/>
                <nd ref="258023391"/>
                <nd ref="736948622"/>
                <nd ref="930330659"/>
                <nd ref="736861978"/>
                <nd ref="930330542"/>
                <nd ref="930330544"/>
                <nd ref="929808660"/>
                <nd ref="736934948"/>
                <nd ref="930330644"/>
                <nd ref="736871567"/>
                <nd ref="619628331"/>
                <nd ref="740363293"/>
                <nd ref="931468900"/>
                <tag k="name" v="North Wabash Avenue"/>
                <tag k="highway" v="tertiary"/>
                <tag k="loc_ref" v="44 E"/>
        </way>"
``


My tentative idea is to match up the lat/lons with OSM id using IMPOSM, then find the `nd refs` in the original data and add a property that contains bumpiness information.

In [ ]:
rides, readings = data_munging.read_raw_data()
# This is here temporarily both to create a static dataset and because
# some of the newer android readings are bogus and need to be removed.
rides = rides.loc[rides['created_at'] < '2016-03-01', :]
readings = readings.loc[readings['created_at'] < '2016-03-01', :]

In [ ]:
readings = data_munging.clean_readings(readings)
readings = data_munging.add_proj_to_readings(readings, data_munging.NAD83)

You can get the IP address by linking up to the Docker container running OSRM and pinging it.

In [ ]:
url = 'http://162.243.23.60/osrm-chi-vanilla/'
url = 'http://172.17.0.2:5000/'
nearest_request = url + 'nearest?loc={0},{1}'
match_request = url + 'match?loc={0},{1}&t={2}&loc={3},{4}&t={5}'

In [ ]:
def readings_to_match_str(readings):
    data_str = '&loc={0},{1}&t={2}'
    output_str = ''
    elapsed_time = 0
    for i, reading in readings.iterrows():
        elapsed_time += 1
        new_str = data_str.format(str(reading['start_lat']), str(reading['start_lon']), str(elapsed_time))
        output_str += new_str
    return url + 'match?' + output_str[1:]

In [ ]:
readings_to_match_str(readings.loc[readings['ride_id'] == 128,  :])

In [ ]:
matched_ride = requests.get(readings_to_match_str(readings.loc[readings['ride_id'] == 128,  :])).json()

In [ ]:
snapped_points =  pd.DataFrame(matched_ride['matchings'][0]['matched_points'], columns=['lat', 'lon'])

In [ ]:
ax = snapped_points.plot(x='lon', y='lat', kind='scatter')
readings.loc[readings['ride_id'] == 128,  :].plot(x='start_lon', y='start_lat', kind='scatter', ax=ax)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
a_reading = readings.loc[0, :]
test_match_request = match_request.format(a_reading['start_lat'],
                                      a_reading['start_lon'], 
                                      0,
                                      a_reading['end_lat'],
                                      a_reading['end_lon'],
                                      1)
# This does not work because OSRM does not accept floats as times. 
# test_map_request = map_request.format(*tuple(a_reading[['start_lat', 'start_lon', 'start_time',
#                                                 'end_lat', 'end_lon', 'end_time']]))

In [ ]:
test_nearest_request = nearest_request.format(a_reading['start_lat'], a_reading['start_lon'])

In [ ]:
osrm_response = requests.get(test_match_request).json()
osrm_response['matchings'][0]['matched_points']

In [ ]:
osrm_response = requests.get(test_nearest_request).json()
osrm_response['mapped_coordinate']

In [ ]:
readings['snapped_lat'] = 0
readings['snapped_lon'] = 0

In [ ]:
# total_calls = 0
# for i, reading in readings.iterrows():
#     nearest_request = nearest_request.format(reading['start_lat'], reading['end_lat'])
#     osrm_response = requests.get(osrm_request).json()
#     print osrm_response
#     total_calls += 1
#     if total_calls > 100:
#         break

In [ ]:
# total_calls = 0
# for i, reading in readings.iterrows():
#     data = tuple(reading[['start_lat', 'start_lon']])
# #     print i
# #     print data 
#     osrm_request = nearest_request.format(*data)
#     osrm_response = requests.get(osrm_request).json()
#     readings.loc[i, ['snapped_lat', 'snapped_lon']] = osrm_response['mapped_coordinate']
# #     print osrm_response
#     total_calls += 1
#     if total_calls % 1000 == 0:
#         print(total_calls)

In [ ]:
chi_readings = data_munging.filter_readings_to_chicago(readings)
chi_rides = list(set(chi_readings.ride_id))

In [ ]:
bad_rides = [128, 129, 5.0, 7.0, 131, 133, 34, 169, 16.0]

In [ ]:
for ride_id in chi_rides:
    if ride_id in bad_rides:
        print('ride_id')
        try:
            print('num readings: ' + str(sum(readings['ride_id'] == ride_id)))
        except:
            print('we had some issues here lol')

In [ ]:
# for ride_id in chi_rides:
#     if pd.notnull(ride_id):
#         complete_snapped_points.loc[complete_snapped_points['ride_id'] == ride_id, ['id']] = readings.loc[readings['ride_id'] == ride_id, ['id']]
#         print(complete_snapped_points.loc[complete_snapped_points['ride_id'] == ride_id, ['id']])
#         print(readings.loc[readings['ride_id'] == ride_id, ['id']])

In [ ]:
all_snapped_points = []
readings['snapped_lat'] = np.NaN
readings['snapped_lon'] = np.NaN
for ride_id in chi_rides:
    if pd.notnull(ride_id):
#         for i, reading in readings.loc[readings['ride_id'] == random_ride_id, :].iterrows():
#             ax = plt.plot([reading['start_lon'], reading['end_lon']], [reading['start_lat'], reading['end_lat']])
        ax = readings.loc[readings['ride_id'] == ride_id, :].plot(x='start_lon', y='start_lat')
        try:
            matched_ride = requests.get(readings_to_match_str(readings.loc[readings['ride_id'] == ride_id,  :])).json() 
            readings.loc[readings['ride_id'] == ride_id, ['snapped_lat', 'snapped_lon']] = matched_ride['matchings'][0]['matched_points']
            readings.loc[readings['ride_id'] == ride_id, :].plot(x='snapped_lon', y='snapped_lat', ax=ax)
        except:
            print('could not snap')
        plt.title('Plotting Ride ' + str(ride_id))
        fig = plt.gcf()
        fig.set_size_inches(18.5, 10.5)
        plt.show()

In [ ]:
# all_snapped_points = []
# readings['snapped_lat'] = np.NaN
# readings['snapped_lon'] = np.NaN
# for ride_id in chi_rides:
#     if pd.notnull(ride_id):
# #         for i, reading in readings.loc[readings['ride_id'] == random_ride_id, :].iterrows():
# #             ax = plt.plot([reading['start_lon'], reading['end_lon']], [reading['start_lat'], reading['end_lat']])
#         ax = readings.loc[readings['ride_id'] == ride_id, :].plot(x='start_lon', y='start_lat')
#         try:
#             matched_ride = requests.get(readings_to_match_str(readings.loc[readings['ride_id'] == ride_id,  :])).json() 
#             readings.loc[readings['ride_id'] == ride_id, ['snapped_lat', 'snapped_lon']] = matched_ride['matchings'][0]['matched_points']
#             snapped_points =  pd.DataFrame(matched_ride['matchings'][0]['matched_points'], columns=['lat', 'lon'])
#             readings.loc[readings['ride_id'] == 'ride_id', ['snapped_lat', 'snapped_lon']] = matched_ride['matchings'][0]['matched_points']
#             readings.loc[readings['ride_id'] == 'ride_id', 'snapped_lon'] = snapped_points['lon']
#             snapped_points['ride_id'] = ride_id
#             snapped_points['id'] = readings.loc[readings['ride_id'] == ride_id, ['id']]
#             snapped_points.plot(x='lon', y='lat', ax=ax)
#             print('this thing right here')
#             all_snapped_points.append(snapped_points)
#         except:
#             print('could not snap')
#         plt.title('Plotting Ride ' + str(ride_id))
#         fig = plt.gcf()
#         fig.set_size_inches(18.5, 10.5)
#         plt.show()

In [ ]:
good_chi_rides = [i for i in chi_rides if i not in bad_rides]

In [ ]:
# complete_snapped_points = pd.concat(all_snapped_points)

In [ ]:
ax = readings.loc[readings['ride_id'] == 2, :].plot(x='snapped_lon', y='snapped_lat', style='r-')
for ride_id in good_chi_rides:
    print(ride_id)
    try:
#         readings.loc[readings['ride_id'] == ride_id, :].plot(x='start_lon', y='start_lat', ax=ax)
        readings.loc[readings['ride_id'] == ride_id, :].plot(x='snapped_lon', y='snapped_lat', ax=ax, style='b-')
    except:
        print('bad')
ax = readings.loc[readings['ride_id'] == 2, :].plot(x='snapped_lon', y='snapped_lat', style='r-', ax=ax)
fig = plt.gcf()
fig.set_size_inches(36, 36)
plt.show()

In [ ]:
# lat_lons_by_ride = collections.defaultdict(set)
# for index, row in readings.iterrows():
#     lat_lons_by_ride[tuple(row[['snapped_lat', 'snapped_lon']])].add(row['ride_id'])

In [ ]:
# most_common_lat_lons = sorted(lat_lons_by_ride.items(), key=lambda i: len(i[1]), reverse=True)

This shows that we are in fact snapping to the same points across rides.

In [ ]:
total_printed = 0
for item in most_common_lat_lons:
    total_printed += 1
    print item
    if total_printed > 300:
        break

In [ ]:
readings['next_snapped_lat'] = np.NaN
readings['next_snapped_lon'] = np.NaN
for ride_id in good_chi_rides:
    next_lat_lon = (np.NaN, np.NaN)
    for index, row in reversed(list(readings.loc[readings['ride_id'] == ride_id, :].iterrows())):
        readings.loc[index, ['next_snapped_lat', 'next_snapped_lon']] = next_lat_lon
        if (row['snapped_lat'], row['snapped_lon']) != next_lat_lon:
            next_lat_lon = (row['snapped_lat'], row['snapped_lon'])

In [ ]:
road_bumpiness = collections.defaultdict(list)
for index, reading in readings.iterrows():
    if reading['gps_mph'] < 30 and reading['gps_mph'] > 3:
        osm_segment = [(reading['snapped_lat'], reading['snapped_lon']),
                      (reading['next_snapped_lat'], reading['next_snapped_lon'])]
        osm_segment = sorted(osm_segment)
        if all([lat_lon != (np.NaN, np.NaN) for lat_lon in osm_segment]):
            road_bumpiness[tuple(osm_segment)].append(reading['abs_mean_over_speed'])

In [ ]:
# sorted_road_bumpiness = sorted(road_bumpiness.items(), key=lambda i: len(i[1]), reverse=True)

In [ ]:
agg_road_bumpiness = dict((osm_segment, np.mean(road_bumpiness[osm_segment])) for osm_segment in road_bumpiness)

In [ ]:
with open('../dat/agg_road_bumpiness.txt', 'w+') as f:
    f.write(str(agg_road_bumpiness))

In [ ]:
with open('../dat/agg_road_bumpiness.txt', 'r') as f:
    agg_road_bumpiness = f.read()

In [ ]:
from numpy import nan

In [ ]:
agg_road_bumpiness = eval(agg_road_bumpiness)

In [ ]:
def filter_out_na_from_osm_segments(osm_segment):
    return np.NaN in osm_segment[0] or np.NaN in osm_segment[1]

In [ ]:
agg_road_bumpiness = dict((osm_segment, agg_road_bumpiness[osm_segment]) for osm_segment in agg_road_bumpiness if not filter_out_na_from_osm_segments(osm_segment))

In [ ]:
np.max(agg_road_bumpiness.values())

In [ ]:
len(agg_road_bumpiness)

In [ ]:
plt.hist(agg_road_bumpiness.values())

In [ ]:
import matplotlib.colors as colors

In [ ]:
plasma = cm = plt.get_cmap('plasma')
cNorm  = colors.Normalize(vmin=0, vmax=1.0)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=plasma)

In [ ]:
for osm_segment, bumpiness in agg_road_bumpiness.items():
    plt.plot([osm_segment[0][1], osm_segment[1][1]],
             [osm_segment[0][0], osm_segment[1][0]],
             color=scalarMap.to_rgba(bumpiness))
fig = plt.gcf()
fig.set_size_inches(12, 24)
plt.show()

In [ ]:
# total_calls = 0
# for i, reading in readings.iterrows():
#     data = tuple(reading[['start_lat', 'start_lon']]) + (0,) + tuple(reading[['end_lat', 'end_lon']]) + (1,)   
#     print i
#     print data 
#     osrm_request = map_request.format(*data)
#     osrm_response = requests.get(osrm_request).json()
#     print osrm_response
#     total_calls += 1
#     if total_calls > 100:
#         break

In [ ]:
# total_calls = 0
# for i, reading in readings.iterrows():
#     data = tuple(reading[['start_lat', 'start_lon']]) + (0,) + tuple(reading[['end_lat', 'end_lon']]) + (1,)   
#     print i
#     print data 
#     osrm_request = map_request.format(*data)
#     osrm_response = requests.get(osrm_request).json()
#     print osrm_response
#     total_calls += 1
#     if total_calls > 100:
#         break

In [ ]:
# for i, reading in readings.iterrows():
#     data = tuple(reading[['start_lat', 'start_lon']]) + (0,) + tuple(reading[['end_lat', 'end_lon']]) + (1,)   
#     osrm_request = map_request.format(*data)
#     osrm_response = requests.get(osrm_request).json()
#     print osrm_response
#     readings.loc[i, ['snapped_start_lat', 'snapped_start_lon']] = osrm_response['matchings'][0]['matched_points'][0]
#     break